In [1]:
import pandas as pd
import numpy as np

### Lidando com dados inexistentes
Dados ausentes são comuns na maioria das aplicações de análise de dados. Um dos objetivos do projeto pandas era tornar o trabalho com dados ausentes o mais simples possível.  
O `pandas` usa o valor de ponto flutuante NaN (não é um número) para representar dados ausentes. É apenas usado como um sentinela que pode ser facilmente detectado:

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

O valor None do Python também é entendido como um NaN.

<table width='100%'>
    <thead>
        <tr>
            <th>Método</th>
            <th colspan=3>Descrição</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>dropna</td>
            <td colspan=3>Filtra os eixos procurando por dados inexistentes e realiza a limpeza por meio de exclusão. O limite tolerável de dados nulos pode ser informado por parâmetro.</td>
        </tr>
        <tr>
            <td>fillna</td>
            <td colspan=3>Preenche os dados ausentes com algum valor ou usa um método de interpolação, como 'ffill' ou 'bfill'.</td>
        </tr>
        <tr>
            <td>isnull</td>
            <td colspan=3>Retorna um valor booleano para verificação se o elemento é nulo ou não.</td>
        </tr>
        <tr>
            <td>notnull</td>
            <td colspan=3>Negação do `isnull`</td>
        </tr>
    </tbody>
</table>

#### Filtrando dados inexistentes
Você tem várias opções para filtrar dados ausentes. `dropna` pode ser muito útil. Em uma série, ele retorna a série com apenas
os dados não nulos e os valores do índice:

In [4]:
from numpy import nan as NA

In [6]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [7]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Com os objetos DataFrame, eles são um pouco mais complexos. Você pode remover linhas ou colunas que são todas NA ou apenas aquelas que contêm NAs. dropna por padrão remove qualquer linha que contenha um valor ausente:

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Remover colunas pode ser feito com as mesma função mudando o eixo para 1.

In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Uma maneira relacionada de filtrar as linhas do DataFrame tende a dizer respeito a dados de séries temporais. Suponha você deseja manter apenas linhas contendo um certo número de observações. Você pode indicar isso com o argumento thresh:

In [14]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,-0.343788,-0.933268,0.087674
1,-1.048404,0.427257,0.297429
2,2.177429,0.470734,-0.498619
3,0.121666,0.436753,0.187406
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


In [15]:
df.iloc[:4, 1] = NA; df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.343788,NaN,NaN
1,-1.048404,NaN,NaN
2,2.177429,NaN,-0.498619
3,0.121666,NaN,0.187406
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


In [16]:
df.dropna(thresh=3)

,0,1,2
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


#### Preenchimento em dados inexistentes
Em vez de filtrar os dados ausentes (e potencialmente descartar outros dados), você pode preencher os “buracos” de várias maneiras. Para a maioria dos propósitos, o método `fillna` é a função a ser usada. Chamar o fillna com uma constante substitui
valores ausentes com esse valor:

In [17]:
df.fillna(0)

,0,1,2
0,-0.343788,0.000000,0.000000
1,-1.048404,0.000000,0.000000
2,2.177429,0.000000,-0.498619
3,0.121666,0.000000,0.187406
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


Chamando fillna com um dict, você pode usar um valor de preenchimento diferente para cada coluna:

In [18]:
df.fillna({1: 0.5, 3: -1})

,0,1,2
0,-0.343788,0.500000,NaN
1,-1.048404,0.500000,NaN
2,2.177429,0.500000,-0.498619
3,0.121666,0.500000,0.187406
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


fillna retorna um novo objeto, mas você pode modificar o objeto existente no local:

In [19]:
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.343788,0.000000,0.000000
1,-1.048404,0.000000,0.000000
2,2.177429,0.000000,-0.498619
3,0.121666,0.000000,0.187406
4,1.542115,-0.222170,-0.157499
5,0.433952,-0.177334,0.937263
6,-0.942995,0.175568,0.029088


Os mesmos métodos de interpolação disponíveis para reindexação podem ser usados com fillna.